<a href="https://colab.research.google.com/github/plotia/Computational-Methods-for-Biological-Modelling-and-Simulation/blob/main/HW6_02_712_Q4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import numpy as np

def exp_max(m, n, E, C, x, r):

  # Initialize λ with equal frequencies
    λ = np.ones(m) / m

  # Create matrix y for taking values yij
    y = np.zeros((m, n))

  # Prepare numerators and denominators for y
    Num = np.zeros((n, m))

    for _ in range(r):

      for i in range(n):
        for j in range(m):
          Num[i][j] = λ[j] * E[j][i]

      for i in range (n):
        for j in range(m):
          y[j][i] = Num[i][j] * x[i] / np.sum(Num[i])

      for j in range(m):
        λ[j] = np.sum(y[j]) / (n * C)

    return λ, y

In [36]:
# Test data
m, n = 3, 4
E = np.array([
    [1, 0, 1, 1],
    [1, 1, 0, 0],
    [0, 1, 1, 0]
])
C = 50
x = np.array([24, 16, 32, 19])
r = 100


In [37]:
exp_max(m,n,E,C,x,r)

(array([0.33025005, 0.00945263, 0.11529732]),
 array([[23.32853607,  0.        , 23.72147338, 19.        ],
        [ 0.67146393,  1.21906204,  0.        ,  0.        ],
        [ 0.        , 14.78093796,  8.27852662,  0.        ]]))

In [45]:
import numpy as np

def exp_max_results(m, n, E, C, x, r):

  # Initialize λ with equal frequencies
    λ = np.ones(m) / m

  # Create matrix y for taking values yij
    y = np.zeros((m, n))

  # Prepare numerators and denominators for y
    Num = np.zeros((n, m))

    for _ in range(r):

      for i in range(n):
        for j in range(m):
          Num[i][j] = λ[j] * E[j][i]

      for i in range (n):
        for j in range(m):
          y[j][i] = Num[i][j] * x[i] / np.sum(Num[i])

      for j in range(m):
        λ[j] = np.sum(y[j]) / (n * C)

    print(f"Strain {np.argmax(λ)+1} is the most common strain.")

In [46]:
exp_max_results(m,n,E,C,x,r)

Strain 1 is the most common strain.
